# Candidate generation

In [3]:
import zipfile
from tqdm import tqdm
import orc

ocr = orc.ocr.GCPVision()
documents = []

with zipfile.ZipFile('data/invoices/files.zip') as z:
    for invoice in tqdm(list((zipfile.Path(z) / "files").iterdir())[:100]):
        content = invoice.read_bytes()
        document = ocr(content, key=invoice.name)
        documents.append(document)

100%|██████████| 100/100 [00:06<00:00, 14.35it/s]


In [7]:
def named(name, pattern):
    return f"(?P<{name}>{pattern})"

def any_of(*patterns):
    return "(" + "|".join(patterns) + ")"


YYYY = named("YYYY", r"\d{4}")
MM = named("MM", r"\d{1,2}")
DD = named("DD", r"\d{1,2}")
SEP = any_of(r'\-', r'\s', '')


class YYYYMMDD:

    @property
    def pattern(self):
        return f'{YYYY}{SEP}{MM}{SEP}{DD}'

    def parse(self, raw):
        return dt.date(
            year=int(raw.group('YYYY')),
            month=int(raw.group('MM')),
            day=int(raw.group('DD'))
        )

In [15]:
for candidate in orc.candidates.search(documents[0], YYYYMMDD().pattern):
    print(candidate)
    print()

69000356

 349 783
S
(349 783


944 0003
S
(944 0003

 69002
S
(69002

 69002
S
(69002

04 78 38
  S
04P78 38

09/2011
  S
09P2011



## TODO

The above doesn't work correctly, due to the way the `NearMatch` class works. The list of edits is not correct.